In [14]:
from openeye import oechem, oeshape, oequacpac, oeomega, oegrid
from IPython.display import Image
from openeye import oedepict
import time

In [15]:
def ChangeAtomCav(cav):
    """Replace Atom cavity Type and Isotope"""
    for atom in cav.GetAtoms():
        coords = oechem.OEFloatArray(3)
        cav.GetCoords(atom, coords)
        # Hydrophobic cavity points
        if atom.GetName() == 'CA':
            atom.SetType('C.3')
            atom.SetAtomicNum(6)
            atom.SetIsotope(13)
            atom.SetImplicitHCount(0)
        # Aromatic cavity points
        elif atom.GetName() == 'CZ':
            atom.SetType('C.ar')
            atom.SetAtomicNum(6)
            atom.SetIsotope(15)
            atom.SetImplicitHCount(0)
        # Hydrogen-bond donor cavity points
        elif atom.GetName() == 'N':
            atom.SetType('N.am')
            atom.SetAtomicNum(7)
            atom.SetIsotope(14)
            atom.SetImplicitHCount(0)
        # Hydrogen-bond acceptor cavity points
        elif atom.GetName() == 'O':
            atom.SetType('O.2')
            atom.SetAtomicNum(8)
            atom.SetIsotope(14)
            atom.SetImplicitHCount(0)
        # Positively ionizable cavity point
        elif atom.GetName() == 'NZ':
            atom.SetType('N.4')
            atom.SetAtomicNum(7)
            atom.SetIsotope(15)
            atom.SetImplicitHCount(0)
        # Hydrogen-bond acceptor/donor cavity points
        elif atom.GetName() == 'OG':
            atom.SetType('O.3')
            atom.SetAtomicNum(8)
            atom.SetIsotope(15)
            atom.SetImplicitHCount(0)
        # Negatively ionizable cavity points
        elif atom.GetName() == 'OD1':
            atom.SetType('O.co2')
            atom.SetAtomicNum(8)
            atom.SetIsotope(17)
            atom.SetImplicitHCount(0)
        # Metal cavity points
        elif atom.GetName() == 'Zn':
            atom.SetType('Zn')
            atom.SetAtomicNum(30)
            atom.SetIsotope(54)
            atom.SetImplicitHCount(0)
        # Dummy cavity points
        elif atom.GetName() == 'DU':
            atom.SetType('H')
            atom.SetIsotope(2)
            atom.SetAtomicNum(1)
            atom.SetImplicitHCount(0)
    return(cav)

In [16]:
# ICHEM alignment and scoring settings
cff = oeshape.OEColorForceField()
cff.Init("CavLig14.cff", False)

refcav = oechem.OEGraphMol()
reffs = oechem.oemolistream("without_water_ligand_hydrogen_hydrogen.mol2")
oechem.OEReadMolecule(reffs, refcav)
refmol = ChangeAtomCav(refcav)
prep = oeshape.OEOverlapPrep()
prep.SetColorForceField(cff)
prep.Prep(refmol)

colopt = oeshape.OEColorOptions()
colopt.SetColorForceField(cff)
shopt = oeshape.OEShapeOptions()
shopt.SetCarbonRadius(1.7)
options = oeshape.OEOverlayOptions()
options.SetMaxOptSteps(0)
#options.SetStarts(oeshape.OESubrocsStarts())
options.SetStarts(oeshape.OEInertialStarts())
#options.SetOverlapFunc(oeshape.OEOverlapFunc(oeshape.OEExactShapeFunc(shopt), oeshape.OEExactColorFunc(colopt)))
options.SetOverlapFunc(oeshape.OEOverlapFunc(oeshape.OEGridShapeFunc(shopt), oeshape.OEGridColorFunc(colopt)))
overlay = oeshape.OEOverlay(options)
overlay.SetupRef(refmol)

True

In [17]:
# OMEGA settings
oechem.OEThrow.SetLevel(10000)
omega_options = oeomega.OEOmegaOptions()
omega_options.SetStrictStereo(False)
omega_options.SetEnergyWindow(10)
#omega_options.SetMaxConfs(200)
omega_options.SetMaxConfs(100)
omega = oeomega.OEOmega(omega_options)

In [18]:
def ichem_score(oemol, omega=omega, ichem_overlay=overlay):
    best_conf = oechem.OEMol()
    best_score = 0.0
    stereo = False
    oequacpac.OEGetReasonableProtomer(oemol)
    enantiomers = list(oeomega.OEFlipper(oemol.GetActive(), 3, False, True)) 
    for k, enantiomer in enumerate(enantiomers):
        enantiomer = oechem.OEMol(enantiomer)
        ret_code = omega.Build(enantiomer)
        if ret_code == oeomega.OEOmegaReturnCode_Success:
            if k == 0:
                oemol = oechem.OEMol(enantiomer.SCMol())
                oemol.DeleteConfs()
            stereo = True
            for x in enantiomer.GetConfs():
                oemol.NewConf(x)
    if stereo:
        prep = oeshape.OEOverlapPrep()
        prep.Prep(oemol)
        score = oeshape.OEBestOverlayScore()
        ichem_overlay.BestOverlay(score, oemol, oeshape.OEHighestTanimotoCombo())
        outmol = oechem.OEGraphMol(oemol.GetConf(oechem.OEHasConfIdx(score.GetFitConfIdx())))
        score.Transform(outmol)
        oeshape.OERemoveColorAtoms(outmol)
        if oechem.OEHasImplicitHydrogens(outmol):
            oechem.OEAddExplicitHydrogens(outmol)
        best_conf = oechem.OEMol(outmol)
        best_score = score.GetTanimotoCombo()
    return stereo, best_conf, best_score

In [19]:
# Output file
outfs = oechem.oemolostream("ichem_best.sdf")

# Input SMILES
smiles = "c1c(O)ccc(c12)C[C@@H]([NH2+]C2)C(=O)N[C@@H](C(C)C)C[N@H+](CC3)C[C@H](C)[C@]3(C)c4cc(O)ccc4"

# Calculate ICHEM score and save corresponding 3D conformer
fitfs = oechem.OEMol()
if oechem.OESmilesToMol(fitfs, smiles):
    stereo_, best_conf_, best_score_ = ichem_score(fitfs)
    oechem.OEWriteMolecule(outfs, best_conf_)
    print(f'Score: {best_score_}')

Score: 0.45124074816703796
